In [25]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90.0, 90.0, size=1500)
lngs = np.random.uniform(-90.0, 90.0, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
coordinates = list(lat_lngs)

In [4]:
# Create a list of cities from the list of latitude & longitude
cities = []

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    # If the city does not exist in the list, add it to the cities list
    if city not in cities:
        cities.append(city)
len(cities)

582

In [9]:
# URL for Weather Map API Call
OpenWeather_url = "https://api.openweathermap.org/data/2.5/weather?"

In [35]:
# Create an empty list to hold the weather data
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city name
    query_url = OpenWeather_url + "units=imperial" + "&appid=" + weather_api_key + "&q=" + city

    # Log the URL, record and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to record_count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(query_url).json()
        
        # Retrieve required data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_maxtemp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        # Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append data into city_data list
        city_data.append({"City":city.title(),
                         "Lat":city_lat,
                         "Lng":city_lng,
                         "Max Temp":city_maxtemp,
                         "Humidity":city_humidity,
                         "Cloudiness":city_clouds,
                         "Wind Speed":city_wind,
                         "Country":city_country,
                         "Date":city_date})
    
    # In case of error, skip the current city
    except:
        print("City Not Found. Skipping...")
        pass

# Completion of Data Retrieval
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

Beginning Data Retrieval
-----------------------------
Processing Record 1 of Set 1 | comodoro rivadavia
Processing Record 2 of Set 1 | bijar
Processing Record 3 of Set 1 | bereda
Processing Record 4 of Set 1 | soyo
Processing Record 5 of Set 1 | terrace bay
Processing Record 6 of Set 1 | bredasdorp
Processing Record 7 of Set 1 | mar del plata
Processing Record 8 of Set 1 | port alfred
Processing Record 9 of Set 1 | taolanaro
City Not Found. Skipping...
Processing Record 10 of Set 1 | dingle
Processing Record 11 of Set 1 | upernavik
Processing Record 12 of Set 1 | illoqqortoormiut
City Not Found. Skipping...
Processing Record 13 of Set 1 | belmonte
Processing Record 14 of Set 1 | salalah
Processing Record 15 of Set 1 | belushya guba
City Not Found. Skipping...
Processing Record 16 of Set 1 | ejido
Processing Record 17 of Set 1 | hamilton
Processing Record 18 of Set 1 | chuy
Processing Record 19 of Set 1 | cidreira
Processing Record 20 of Set 1 | santa fe
Processing Record 21 of Set 1 |

In [41]:
city_data_df = pd.DataFrame(city_data)
city_data_df = city_data_df[['City','Country','Date','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed']]

In [42]:
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Comodoro Rivadavia,AR,2020-04-04 15:19:36,-45.87,-67.50,59.00,40,0,19.46
1,Bijar,IR,2020-04-04 15:20:11,35.87,47.61,46.71,30,31,16.67
2,Bereda,ES,2020-04-04 15:20:12,43.27,-7.54,71.60,35,87,14.99
3,Soyo,AO,2020-04-04 15:20:12,-6.13,12.37,86.00,84,75,5.82
4,Terrace Bay,CA,2020-04-04 15:20:12,48.78,-87.10,28.40,92,90,9.17
5,Bredasdorp,ZA,2020-04-04 15:20:13,-34.53,20.04,75.20,44,100,21.92
6,Mar Del Plata,AR,2020-04-04 15:20:12,-38.00,-57.56,69.01,64,0,19.46
7,Port Alfred,ZA,2020-04-04 15:20:13,-33.59,26.89,78.01,66,3,9.51
8,Dingle,PH,2020-04-04 15:20:40,11.00,122.67,79.32,77,0,10.63
9,Upernavik,GL,2020-04-04 15:18:05,72.79,-56.15,14.54,80,0,14.43


In [43]:
# Create the output file (csv)
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")